In [94]:
import math
import string

tests = """
3:20
"""

class Range:
    def __init__(self, start, stop=1, step=1):
        self.start = start
        self.stop = stop
        self.step = step
        self.value = None
        
    def evaluate(self):
        self.value = list(range(self.start, self.stop+1, self.step))
        return self.value

class Token:
    def __init__(self, string='', token_type=None):
        self.string = string
        self.token_type = self.type = token_type

class Array:
    def __init__(self, terms):
        self.data = []
        for t in terms:
            if type(t) is list:
                self.data.append(array(t))
            else:
                if t not in list(',;'):
                    self.data.append(t)

class Block:
    def __init__(self, components=None, parsed=None, parser=None, r=None):
        self.components = components
        self.parser = parser
        self.children = []
        
        if self.components:
            comp_vals = []
            for q in self.components:
                if type(q) is Token:
                    print(True)
                    q = q.string
                if q.isnumeric():
                    q = int(q)
                comp_vals.append(q)
            self.parsed = parser(*r(comp_vals))
            
    def add(self, x):
        self.children.append(x)
        
    def __getitem__(self, i):
        return self.children[i]
                    
class Program:
    def __init__(self, source):
        self.source = source
        self.tree = Block()
        
        self.char_sets = {
            'op': '!@#$%^&*/-+<>',
            'syntax': '()[]{},.;:=|',
            'letter': string.ascii_lowercase,
            'numeric': string.digits + '.'
        }
        self.patterns = {
#             'range': ['numeric', ':', 'numeric']
            'range': [lambda x: [x[0].type == 'numeric' and x[1] == ':' and x[2].type == 'numeric'], lambda x: [x[0], x[2]], Range]
        }
        
        self.parse()
        
    def char_type(self, x):
#         return list(filter(lambda x: k for k, v in self.char_sets if x in v))[0]
        return [k for k, v in self.char_sets.items() if x in v][0]
        
    def parse(self):
        statements = self.source.replace('|', '\n').split('\n')
        statements = list(filter(None, statements))
        for s in statements:
            statement_parse = []
            token = Token(s[0], self.char_type(s[0]))
            for c in s[1:]:
                c_type = self.char_type(c)
                if c_type == token.token_type:
                    token.string += c
                else:
                    statement_parse.append(token)
                    token = Token(c, c_type)
            statement_parse.append(token)
            self.tree.add(statement_parse)
        for i, b in enumerate(self.tree.children):
            for k, v in self.patterns.items():
                pattern, result, block_type = v
                if pattern(b):
                    self.tree.children[i] = Block(b, parser=block_type, r=result)
                    
    def execute(self, node=None):
        if not node:
            node = self.tree
        
        for b in node:
            if b.parser in [Range]:
                b.parsed.evaluate()
            else:
#                 for v in b:
#                     self.execute()
                self.execute(node=b)
                    
            
    
        
M = Program(tests)
M.execute()
vars(M.tree[0].parsed)

True
True
True


{'start': 3,
 'stop': 20,
 'step': 1,
 'value': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}